In [ ]:
import PyPDF2
import re
 
#path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
filename = r"\files\factsheet-march-2022.pdf"
 
read_file = PyPDF2.PdfReader(samco_path)
 
num_pages = len(read_file.pages)
data_all = []
 
i = 1
for i in range(num_pages):
    files = read_file.pages[i]
    data = files.extract_text()
    i += 1
    print(f"page number : {i}")
    data_all.append(data)
 

In [ ]:
import pprint
pprint.pprint(data_all[12])